In [ ]:
# install java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# install spark (change the version number if needed)
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz

# unzip the spark file to the current folder
!tar xf spark-3.0.0-bin-hadoop3.2.tgz

# set your spark folder to your system path environment. 
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"


# install findspark using pip
!pip install -q findspark

!pip install pyspark

import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 45 kB/s 
     |████████████████████████████████| 199 kB 55.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=ab352e51163e638230309ca2a4297b62be76931341cb841a669871eaa17a9cb6
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


# **Labs 1 and 2 PySpark:**

In these labs we will be using the "[[NeurIPS 2020] Data Science for COVID-19 (DS4C)](https://www.kaggle.com/datasets/kimjihoo/coronavirusdataset?select=PatientInfo.csv)" dataset, retrieved from [Kaggle](https://www.kaggle.com/) on 1/6/2022, for educational non commercial purpose, License
[CC BY-NC-SA 4.0
](https://creativecommons.org/licenses/by-nc-sa/4.0/)


The csv file that we will be using in this lab is **PatientInfo**.

## PatientInfo.csv

**patient_id**
the ID of the patient

**sex**
the sex of the patient

**age**
the age of the patient

**country**
the country of the patient

**province**
the province of the patient

**city**
the city of the patient

**infection_case**
the case of infection

**infected_by**
the ID of who infected the patient


**contact_number**
the number of contacts with people

**symptom_onset_date**
the date of symptom onset

**confirmed_date**
the date of being confirmed

**released_date**
the date of being released

**deceased_date**
the date of being deceased

**state**
isolated / released / deceased

### Import the pyspark and check it's version

In [ ]:
spark.version

'3.0.0'

### Import and create SparkSession

In [ ]:
spark = SparkSession.builder.getOrCreate()


### Load the PatientInfo.csv file and show the first 5 rows

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

In [ ]:
df = spark.read.option("header", "true").csv("/content/drive/MyDrive/PatientInfo.csv" , inferSchema=True)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df.show(5)

+----------+------+---+-------+--------+-----------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+
|patient_id|   sex|age|country|province|       city|      infection_case|infected_by|contact_number|symptom_onset_date|confirmed_date|released_date|deceased_date|   state|
+----------+------+---+-------+--------+-----------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+
|1000000001|  male|50s|  Korea|   Seoul| Gangseo-gu|     overseas inflow|       null|            75|        2020-01-22|    2020-01-23|   2020-02-05|         null|released|
|1000000002|  male|30s|  Korea|   Seoul|Jungnang-gu|     overseas inflow|       null|            31|              null|    2020-01-30|   2020-03-02|         null|released|
|1000000003|  male|50s|  Korea|   Seoul|  Jongno-gu|contact with patient| 2002000001|            17|              null|    2020-01-30|   202

### Display the schema of the dataset

In [ ]:
df.printSchema()

root
 |-- patient_id: long (nullable = true)
 |-- sex: string (nullable = true)
 |-- age: string (nullable = true)
 |-- country: string (nullable = true)
 |-- province: string (nullable = true)
 |-- city: string (nullable = true)
 |-- infection_case: string (nullable = true)
 |-- infected_by: string (nullable = true)
 |-- contact_number: string (nullable = true)
 |-- symptom_onset_date: string (nullable = true)
 |-- confirmed_date: string (nullable = true)
 |-- released_date: string (nullable = true)
 |-- deceased_date: string (nullable = true)
 |-- state: string (nullable = true)



### Display the statistical summary

In [ ]:
df.describe().show()

+-------+--------------------+------+----+----------+--------+--------------+--------------------+--------------------+--------------------+------------------+--------------+-------------+-------------+--------+
|summary|          patient_id|   sex| age|   country|province|          city|      infection_case|         infected_by|      contact_number|symptom_onset_date|confirmed_date|released_date|deceased_date|   state|
+-------+--------------------+------+----+----------+--------+--------------+--------------------+--------------------+--------------------+------------------+--------------+-------------+-------------+--------+
|  count|                5165|  4043|3785|      5165|    5165|          5071|                4246|                1346|                 791|               690|          5162|         1587|           66|    5165|
|   mean|2.8636345618679576E9|  null|null|      null|    null|          null|                null|2.2845944015643125E9|1.6772572523506988E7|            

### Using the state column.
### How many people survived (released), and how many didn't survive (isolated/deceased)?

In [ ]:
# release_isolated = df.groupBy('state').count()
# release_isolated.select("count").show()
print("no of released : ",df.filter('state = "released" ').count())
print("no of isolated/deceased : ",df.filter('state != "released" ').count())


no of released :  2929
no of isolated/deceased :  2236


In [ ]:
df.groupBy('state').count().show()

+--------+-----+
|   state|count|
+--------+-----+
|isolated| 2158|
|released| 2929|
|deceased|   78|
+--------+-----+



### Display the number of null values in each column

In [ ]:
from pyspark.sql.functions import col,isnan, when, count
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]
   ).show()

+----------+----+----+-------+--------+----+--------------+-----------+--------------+------------------+--------------+-------------+-------------+-----+
|patient_id| sex| age|country|province|city|infection_case|infected_by|contact_number|symptom_onset_date|confirmed_date|released_date|deceased_date|state|
+----------+----+----+-------+--------+----+--------------+-----------+--------------+------------------+--------------+-------------+-------------+-----+
|         0|1122|1380|      0|       0|  94|           919|       3819|          4374|              4475|             3|         3578|         5099|    0|
+----------+----+----+-------+--------+----+--------------+-----------+--------------+------------------+--------------+-------------+-------------+-----+



## Data preprocessing

### Fill the nulls in the deceased_date with the released_date. 
- You can use <b>coalesce</b> function

In [ ]:
from pyspark.sql.functions import coalesce
    
df = df.withColumn("deceased_date",coalesce(df.released_date,df.deceased_date))

In [ ]:
df.show(5)

+----------+------+---+-------+--------+-----------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+
|patient_id|   sex|age|country|province|       city|      infection_case|infected_by|contact_number|symptom_onset_date|confirmed_date|released_date|deceased_date|   state|
+----------+------+---+-------+--------+-----------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+
|1000000001|  male|50s|  Korea|   Seoul| Gangseo-gu|     overseas inflow|       null|            75|        2020-01-22|    2020-01-23|   2020-02-05|   2020-02-05|released|
|1000000002|  male|30s|  Korea|   Seoul|Jungnang-gu|     overseas inflow|       null|            31|              null|    2020-01-30|   2020-03-02|   2020-03-02|released|
|1000000003|  male|50s|  Korea|   Seoul|  Jongno-gu|contact with patient| 2002000001|            17|              null|    2020-01-30|   202

### Add a column named no_days which is difference between the deceased_date and the confirmed_date then show the top 5 rows. Print the schema.
- <b> Hint: You need to typecast these columns as date first <b>

In [ ]:
from pyspark.sql.types import DoubleType, DateType
from pyspark.sql.functions import *
df = df.withColumn("deceased_date", df["deceased_date"].cast(DateType()).alias("deceased_date"))
df = df.withColumn("confirmed_date", df["confirmed_date"].cast(DateType()).alias("confirmed_date"))

# df = df.withColumn("no_days", df["deceased_date"] - df["confirmed_date"] )


In [ ]:
df = df.withColumn("no_days", df["deceased_date"] - df["confirmed_date"] ).alias("no_days")

In [ ]:
df.show(2)

+----------+----+---+-------+--------+-----------+---------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+---------------+
|patient_id| sex|age|country|province|       city| infection_case|infected_by|contact_number|symptom_onset_date|confirmed_date|released_date|deceased_date|   state|        no_days|
+----------+----+---+-------+--------+-----------+---------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+---------------+
|1000000001|male|50s|  Korea|   Seoul| Gangseo-gu|overseas inflow|       null|            75|        2020-01-22|    2020-01-23|   2020-02-05|   2020-02-05|released|        13 days|
|1000000002|male|30s|  Korea|   Seoul|Jungnang-gu|overseas inflow|       null|            31|              null|    2020-01-30|   2020-03-02|   2020-03-02|released|1 months 2 days|
+----------+----+---+-------+--------+-----------+---------------+-----------+--------------+--

### Add a is_male column if male then it should yield true, else then False

In [ ]:
from pyspark.sql import functions as f
from pyspark.sql.functions import when


df = df.withColumn('is_male',f. when((df["sex"] == "male"), 1).otherwise(0))
df.show(2)


+----------+----+---+-------+--------+-----------+---------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+---------------+-------+
|patient_id| sex|age|country|province|       city| infection_case|infected_by|contact_number|symptom_onset_date|confirmed_date|released_date|deceased_date|   state|        no_days|is_male|
+----------+----+---+-------+--------+-----------+---------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+---------------+-------+
|1000000001|male|50s|  Korea|   Seoul| Gangseo-gu|overseas inflow|       null|            75|        2020-01-22|    2020-01-23|   2020-02-05|   2020-02-05|released|        13 days|      1|
|1000000002|male|30s|  Korea|   Seoul|Jungnang-gu|overseas inflow|       null|            31|              null|    2020-01-30|   2020-03-02|   2020-03-02|released|1 months 2 days|      1|
+----------+----+---+-------+--------+-----------+-----

### Add a is_dead column if patient state is not released then it should yield true, else then False

- Use <b>UDF</b> to perform this task. 
- However, UDF is not recommended there is no built in function can do the required operation.
- UDF is slower than built in functions.

In [ ]:
df = df.withColumn('is_dead',f. when((df["state"] != "released"), 1).otherwise(0))
df.show(2)

+----------+----+---+-------+--------+-----------+---------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+---------------+-------+-------+
|patient_id| sex|age|country|province|       city| infection_case|infected_by|contact_number|symptom_onset_date|confirmed_date|released_date|deceased_date|   state|        no_days|is_male|is_dead|
+----------+----+---+-------+--------+-----------+---------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+---------------+-------+-------+
|1000000001|male|50s|  Korea|   Seoul| Gangseo-gu|overseas inflow|       null|            75|        2020-01-22|    2020-01-23|   2020-02-05|   2020-02-05|released|        13 days|      1|      0|
|1000000002|male|30s|  Korea|   Seoul|Jungnang-gu|overseas inflow|       null|            31|              null|    2020-01-30|   2020-03-02|   2020-03-02|released|1 months 2 days|      1|      0|
+----------+---

### Change the ages to bins from 10s, 0s, 10s, 20s,.etc to 0,10, 20

In [ ]:
df.printSchema()

root
 |-- patient_id: long (nullable = true)
 |-- sex: string (nullable = true)
 |-- age: string (nullable = true)
 |-- country: string (nullable = true)
 |-- province: string (nullable = true)
 |-- city: string (nullable = true)
 |-- infection_case: string (nullable = true)
 |-- infected_by: string (nullable = true)
 |-- contact_number: string (nullable = true)
 |-- symptom_onset_date: string (nullable = true)
 |-- confirmed_date: date (nullable = true)
 |-- released_date: string (nullable = true)
 |-- deceased_date: date (nullable = true)
 |-- state: string (nullable = true)
 |-- no_days: interval (nullable = true)
 |-- is_male: integer (nullable = false)
 |-- is_dead: integer (nullable = false)



In [ ]:
df = df.withColumn('age', regexp_replace('age','s', ''))

In [ ]:
df.select("age").show(5)

+---+
|age|
+---+
| 50|
| 30|
| 50|
| 20|
| 20|
+---+
only showing top 5 rows



In [ ]:
# df.printSchema()

In [ ]:
df.show(5)

+----------+------+---+-------+--------+-----------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+---------------+-------+-------+
|patient_id|   sex|age|country|province|       city|      infection_case|infected_by|contact_number|symptom_onset_date|confirmed_date|released_date|deceased_date|   state|        no_days|is_male|is_dead|
+----------+------+---+-------+--------+-----------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+---------------+-------+-------+
|1000000001|  male| 50|  Korea|   Seoul| Gangseo-gu|     overseas inflow|       null|            75|        2020-01-22|    2020-01-23|   2020-02-05|   2020-02-05|released|        13 days|      1|      0|
|1000000002|  male| 30|  Korea|   Seoul|Jungnang-gu|     overseas inflow|       null|            31|              null|    2020-01-30|   2020-03-02|   2020-03-02|released|1 months 2 da

### Change age, and no_days  to be typecasted as Double

In [ ]:
# df = df.withColumn("no_days", df["no_days"].cast(DoubleType()).alias("no_days"))
df = df.withColumn("age", df["age"].cast(DoubleType()).alias("age"))

In [ ]:
df = df.withColumn("no_days", f.datediff(f.col("deceased_date"),f.col("confirmed_date")))
df = df.withColumn("no_days", df["no_days"].cast(DoubleType()).alias("no_days"))
df.printSchema()

root
 |-- patient_id: long (nullable = true)
 |-- sex: string (nullable = true)
 |-- age: double (nullable = true)
 |-- country: string (nullable = true)
 |-- province: string (nullable = true)
 |-- city: string (nullable = true)
 |-- infection_case: string (nullable = true)
 |-- infected_by: string (nullable = true)
 |-- contact_number: string (nullable = true)
 |-- symptom_onset_date: string (nullable = true)
 |-- confirmed_date: date (nullable = true)
 |-- released_date: string (nullable = true)
 |-- deceased_date: date (nullable = true)
 |-- state: string (nullable = true)
 |-- no_days: double (nullable = true)
 |-- is_male: integer (nullable = false)
 |-- is_dead: integer (nullable = false)



### Drop the columns
["patient_id","sex","infected_by","contact_number","released_date","state",
"symptom_onset_date","confirmed_date","deceased_date","country","no_days",
"city","infection_case"]

In [ ]:
df = df.drop("patient_id","sex","infected_by","contact_number","released_date","state", "symptom_onset_date","confirmed_date","deceased_date","country","no_days", "city","infection_case")
    

In [ ]:
df.show(2)

+----+--------+-------+-------+
| age|province|is_male|is_dead|
+----+--------+-------+-------+
|50.0|   Seoul|      1|      0|
|30.0|   Seoul|      1|      0|
+----+--------+-------+-------+
only showing top 2 rows



### Recount the number of nulls now

## Now do the same but using SQL select statement

### From the original Patient DataFrame, Create a temporary view (table).

### Use SELECT statement to select all columns from the dataframe and show the output.

### *Using SQL commands*, limit the output to only 5 rows 

### Select the count of males and females in the dataset

### How many people did survive, and how many didn't?

### Now, let's perform some preprocessing using SQL:
1. Convert *age* column to double after removing the 's' at the end -- *hint: check SUBSTRING method*
2. Select only the following columns: `['sex', 'age', 'province', 'state']`
3. Store the result of the query in a new dataframe

## Machine Learning 
### Create a pipeline model to predict is_dead and evaluate the performance.
- Use <b>StringIndexer</b> to transform <b>string</b> data type to indices.
- Use <b>OneHotEncoder</b> to deal with categorical values.
- Use <b>Imputer</b> to fill missing data with mean.

In [ ]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder , VectorAssembler, Imputer

In [ ]:
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]
   ).show()

+----+--------+-------+-------+
| age|province|is_male|is_dead|
+----+--------+-------+-------+
|1380|       0|      0|      0|
+----+--------+-------+-------+



In [ ]:
trainDF , testDF = df.randomSplit([0.8 , 0.2] , seed = 42)

In [ ]:
stringindexer = StringIndexer(inputCols=["province"] , outputCols=["province_index"] , handleInvalid= 'skip')
oheEncoder = OneHotEncoder(inputCols=["province_index"] , outputCols=["province_ohe"])
imputer = Imputer(inputCols=["age"],outputCols=["age"] )

In [ ]:
ass_arr = ["province_ohe" , "age" , "is_male" ]

In [ ]:
vassembler = VectorAssembler(inputCols=ass_arr , outputCol="features")

In [ ]:
# vecTrainDF = vassembler.transform(trainDF)

In [ ]:
from pyspark.ml.classification import RandomForestClassifier

rf = RandomForestClassifier(featuresCol = 'features', labelCol = 'is_dead' , predictionCol="predictions")


In [ ]:
mystages = [stringindexer , oheEncoder,imputer , vassembler ,rf]
from pyspark.ml import Pipeline 

In [ ]:
pip_model = Pipeline(stages=mystages)

In [ ]:
model = pip_model.fit(trainDF)

In [ ]:
pred = model.transform(testDF)

In [ ]:
pred.printSchema()

root
 |-- age: double (nullable = true)
 |-- province: string (nullable = true)
 |-- is_male: integer (nullable = false)
 |-- is_dead: integer (nullable = false)
 |-- province_index: double (nullable = false)
 |-- province_ohe: vector (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- predictions: double (nullable = false)



In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator(predictionCol="predictions", labelCol="is_dead", metricName="rmse")
evaluator.evaluate(pred)

0.40640517364468326